# North Star Metric Tool

This application provides a user-friendly interface to generate North Star Metrics and supporting metrics based on inputs provided. It leverages the power of OpenAI's GPT model and the PromptLayer API to generate these strategies.

## Import necessary libraries and modules

In [ ]:
from dotenv import load_dotenv
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import PromptLayerChatOpenAI
from langchain.output_parsers import PydanticOutputParser
import os
import promptlayer
from pydantic import BaseModel, Field

## Load environment variables

In [ ]:
load_dotenv()

## Set API Keys

In [ ]:
promptlayer.api_key = os.environ.get("PROMPTLAYER_API_KEY")
openai = promptlayer.openai
openai.api_key = os.environ.get("OPENAI_API_KEY")

## Define GPT model and PromptLayer Tag

In [ ]:
GPT_MODEL = "gpt-4"
TEMPERATURE = 0.7
PROMPTLAYER_TAG="NotificationsMatrixGenerator"

## Define the Desired Data Structure Output

In [ ]:
class Metric(BaseModel):
    Category: str = Field(description="The category of the metric")
    Name: str = Field(description="The name of the metric")
    Justification: str = Field(description="The justification of the metric")
    Impact: str = Field(description="The mid/long-term impact of the metric")
    
class ListOfMetrics(BaseModel):
    metrics: list[Metric] = Field(description="List of metrics")

## Set up the parser + inject instructions into the prompt template

In [ ]:
parser = PydanticOutputParser(pydantic_object=ListOfMetrics)
format_instructions = parser.get_format_instructions()

## Generate LLM Response from Template

In [ ]:
def generate_llm_response_from_template(template, **kwargs):
    prompt_template = PromptTemplate(
        input_variables=list(kwargs.keys()),
        template=template,
        partial_variables={"format_instructions": format_instructions}
    )
    llm = PromptLayerChatOpenAI(model=GPT_MODEL, temperature=TEMPERATURE, pl_tags=[PROMPTLAYER_TAG])
    llm_chain = LLMChain(llm=llm, prompt=prompt_template)
    response = llm_chain.predict(**kwargs)
    return response

## Define Prompt Template

In [ ]:
PROMPT="""The North Star Playbook summary says the following: {north_start_plybook_summary}

Given your role as '{user_role}' and with insights from the 'Amplitude North Star Playbook', considering the product category '{product_category}', the core value proposition '{core_value_proposition}', the leading indicator of value '{leading_indicator_of_value}', and the expected user engagement '{expected_user_engagement}', identify the most aligned North Star Metric reflecting user engagement and value realization, the relevant input metrics (2-3) in the categories of Breadth, Depth, and Frequency contributing to this North Star Metric, and the Mid/Long-term Impact on the business as this metric improves.

{format_instructions}"""
PLAYBOOK_CONTEXT="""The Amplitude North Star Playbook provides detailed insights, guidelines, and examples for developing North Star Metrics and their associated input metrics. Here are some key points from the playbook:

North Star Metric: The North Star Metric is a leading indicator of sustainable business results and customer value. It should reflect the heart of your product strategy and the value you provide to customers.

Inputs: Inputs are the factors that, together, produce the North Star Metric. They are as important as the metric itself. Inputs should be influential, complementary factors that you believe most directly affect the North Star Metric and that you can influence through your product offering.

Identifying Inputs: To identify Inputs, you can use techniques like mind-mapping and collaborative brainstorming. Start with your North Star Metric and note relationships and concepts that contribute to it. Inputs should have both a name and a definition.

Justifying Inputs: Inputs should be things that your team can do something about. They should be actionable and within your control. It's better to focus on moving the Inputs that roll up to the North Star Metric rather than trying to directly move the North Star Metric itself.

Aligning with Core Value Propositions: The North Star Metric and its Inputs should align with your core value propositions and reflect the value you provide to customers. They should be a result of the combination of things that your team can do something about.

Leading Indicators of Value: The North Star Metric is a leading indicator of sustainable business results and customer value. As you see the North Star Metric change, you can expect your business results to change accordingly. The Inputs should be the influential factors that directly affect the North Star Metric and can be influenced through your product offering."""

## Get Input parameters to run the model

Example inputs:
- **User Role**: Product Manager
- **Product Category**: Food & Drink (Restaurant Reservations)
- **Core Value Proposition**: "OpenTable empowers users to discover and book the perfect table every time."
- **Leading Indicator of Value**: "User bookings and successful dining experiences."
- **Expected User Engagement**: "Frequent interactions, restaurant searches, and bookings."

In [ ]:
user_role = input("Enter the User Role: ").strip()
product_category = input("Enter the Product Category: ").strip()
core_value_proposition = input("Enter the Core Value Proposition: ").strip()
leading_indicator_of_value = input("Enter the Leading Indicator of Value: ").strip()
expected_user_engagement = input("Enter the Expected User Engagement: ").strip()

## Generate response based on inputs

In [ ]:
response = generate_llm_response_from_template(
            template=PROMPT,
            north_start_plybook_summary=PLAYBOOK_CONTEXT,
            user_role=user_role,
            product_category=product_category,
            core_value_proposition=core_value_proposition,
            leading_indicator_of_value=leading_indicator_of_value,
            expected_user_engagement=expected_user_engagement)
print(response)